In [5]:
from LightGlue.lightglue import LightGlue, SuperPoint, DISK
from LightGlue.lightglue.utils import load_image, rbd
import shutil


Downloading: "https://github.com/cvg/LightGlue/releases/download/v0.1_arxiv/superpoint_lightglue.pth" to C:\Users\Admin/.cache\torch\hub\checkpoints\superpoint_lightglue_v0-1_arxiv.pth
100%|██████████| 45.3M/45.3M [00:02<00:00, 18.2MB/s]
Downloading: "https://raw.githubusercontent.com/cvlab-epfl/disk/master/depth-save.pth" to C:\Users\Admin/.cache\torch\hub\checkpoints\depth-save.pth
100%|██████████| 4.17M/4.17M [00:00<00:00, 17.4MB/s]
Downloading: "https://github.com/cvg/LightGlue/releases/download/v0.1_arxiv/disk_lightglue.pth" to C:\Users\Admin/.cache\torch\hub\checkpoints\disk_lightglue_v0-1_arxiv.pth
100%|██████████| 45.4M/45.4M [00:02<00:00, 20.3MB/s]


In [32]:
def feature_matcher(template_path, image_path, extractor, matcher):
    """
    Function to match features between a template and an image.

    Args:
    template_path (str): Path to the template image.
    image_path (str): Path to the image to match.
    extractor (SuperPoint): SuperPoint extractor.
    matcher (LightGlue): LightGlue matcher.

    Returns:
    int: Number of matched features.
    """

    # load the images
    image0 = load_image(template_path)
    image1 = load_image(image_path)

    # extract local features
    feats0 = extractor.extract(image0)  
    feats1 = extractor.extract(image1)

    # match the features
    matches01 = matcher({'image0': feats0, 'image1': feats1})
    feats0, feats1, matches01 = [rbd(x) for x in [feats0, feats1, matches01]]  # remove batch dimension

    return len(matches01["matches"])

In [34]:
def classify_id_cards(template_path, input_folder_path, green_output_folder, white_output_folder, threshold=800):
    """
    Classify ID cards as green or white based on the template matching result.
    
    Args:
    template_path (str): Path to the template image.
    input_folder_path (str): Path to the input folder.
    green_output_folder (str): Path to the green output folder.
    white_output_folder (str): Path to the white output folder.
    threshold (int): Threshold for the template matching result.

    """

    # SuperPoint+LightGlue
    extractor = SuperPoint(max_num_keypoints=2048).eval()  # load the extractor
    matcher = LightGlue(features='superpoint').eval()  # load the matcher


    for filename in os.listdir(input_folder_path):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            input_path = os.path.join(input_folder_path, filename)

            match_num = feature_matcher(template_path, input_path, extractor, matcher)

            if match_num > threshold:
                print(f'Classified {filename} as green ID card: {match_num} matches')
                output_path = os.path.join(green_output_folder, filename)
            else:
                print(f'Classified {filename} as white ID card: {match_num} matches')
                output_path = os.path.join(white_output_folder, filename)

            shutil.copy(input_path, output_path)

# Test to find the best threshold
template_path = 'template.jpeg'
input_folder_path = 'test'
green_output_folder = 'green_output_folder'
white_output_folder = 'white_output_folder'
classify_id_cards(template_path, input_folder_path, green_output_folder, white_output_folder)

Classified renderfile (12270).jpeg as green ID card: 884 matches
Classified renderfile (12271).jpeg as white ID card: 705 matches
Classified renderfile (12276).jpeg as white ID card: 406 matches
Classified renderfile (12278).jpeg as white ID card: 419 matches
Classified renderfile (12279).jpeg as white ID card: 408 matches
Classified renderfile (12280).jpeg as white ID card: 391 matches
Classified renderfile (12281).jpeg as green ID card: 904 matches
Classified renderfile (12283).jpeg as white ID card: 330 matches
Classified renderfile (12284).jpeg as white ID card: 356 matches
Classified renderfile (12285).jpeg as white ID card: 361 matches
Classified renderfile (12286).jpeg as white ID card: 368 matches
Classified renderfile (12287).jpeg as white ID card: 396 matches
Classified renderfile (12288).jpeg as white ID card: 383 matches
Classified renderfile (12335).jpeg as white ID card: 769 matches
Classified renderfile (12459).jpeg as white ID card: 630 matches
Classified renderfile (12

In [ ]:
# Final classification
threshold = 567
template_path = 'template.jpeg'
input_folder_path = 'input_folder'
green_output_folder = 'green_output_folder_final'
white_output_folder = 'white_output_folder_final'
classify_id_cards(template_path, input_folder_path, green_output_folder, white_output_folder, threshold=threshold)